# Ames Housing - Interpretable ML
- Author: Oliver Mueller
- Last update: 26.01.2024

## Initialize notebook
Load required packages. Set up workspace, e.g., set theme for plotting and initialize the random number generator.

In [ ]:
# Install packages that are not already installed on Colab
#!pip install shap

In [ ]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
import shap

In [ ]:
plt.style.use('fivethirtyeight')

## Problem description

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence. With 76 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this dataset challenges you to predict the final price of each home. More: <https://www.kaggle.com/c/house-prices-advanced-regression-techniques>


## Load data

Load training data from CSV file.

In [ ]:
data_train = pd.read_csv('https://raw.githubusercontent.com/olivermueller/vhbprodok_datascience/main/ames_housing/data/train.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

## Prepare data

Let us first focus on some easy to understand variables.

In [ ]:
data = data[["SalePrice", "LotArea", "GrLivArea", "FullBath", "BedroomAbvGr", "KitchenAbvGr", "OverallQual", "OverallCond"]]

In [ ]:
data.head()

Finally, we will split the data into features (*X*) and labels (*y*) and into training (*X_train, y_train*) and test (*X_test, y_test*) sets.

In [ ]:
X = data.drop("SalePrice", axis=1)
y = data["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Fit a Random Forest model

Since, we are mainly interested in interpreting the model, we will skip feature engineering and hyperparameter tuning and directly jump to fitting a Random Forest model to the trainig data.

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


In [ ]:
pred = rf_model.predict(X_test)
rmse = mean_squared_error(y_test, pred, squared=False)
print(rmse)

The RMSE is not super great, but that is not the main focus of this analysis, as we are interested in interpreting the model. Still, as we can see below, the Random Forest performs much better as linear regression.

In [ ]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
pred = lr_model.predict(X_test)
rmse = mean_squared_error(y_test, pred, squared=False)
print(rmse)

## Global explanations: Permutation-based feature importance

Use `permutation_importance` as model-agnostic function to calculate feature importance.

In [ ]:
r = permutation_importance(rf_model, X_train, y_train, n_repeats=10, random_state=42)

Iterate over the results and put them into a dataframe for easier manipulation and plotting.

In [ ]:
importances = []
for i in r.importances_mean.argsort():
    entry = {}
    entry['feature'] = rf_model.feature_names_in_[i]
    entry['value'] = r.importances_mean[i]
    entry['std'] = r.importances_std[i]
    importances.append(entry)

Sor the dataframe by the mean importance and print it.

In [ ]:
importances = pd.DataFrame(importances).sort_values(by='value', ascending=False)
importances.head(10)

Plot the results as a bar plot.

In [ ]:
sns.barplot(data=importances, x='value', y='feature')
plt.errorbar(data=importances, x='value', y='feature', xerr='std', fmt='none', color='black')
plt.show()

## Global explanations: Partial dependence plots (PDP)

Next, we explore the partial dependence of the response on the features.

In [ ]:
features = ["LotArea"]
PartialDependenceDisplay.from_estimator(rf_model, X_test, features=features)

We can also create 2D partial dependence plots to explore possible interactions between features.

In [ ]:
features = [("LotArea", "GrLivArea")]
PartialDependenceDisplay.from_estimator(rf_model, X_test, features=features)

In [ ]:
features = ["LotArea"]
PartialDependenceDisplay.from_estimator(rf_model, X_test, features=features)

## Local explanations: Individual conditional expectation (ICE) plots

ICE plots are the equivalent of partial dependence plots for individual observations.

In [ ]:
features = ["LotArea"]
PartialDependenceDisplay.from_estimator(rf_model, X_test, features=features, kind='individual')

## Local explanations: SHAP values

Finally, we will calculate SHAP values to explain single predictions from the test set. As we have fitted a Random Forest model, we will use the fast `TreeExplainer` to calculate SHAP values. There are other methods available for other types of learners (e.g., Neural Networks).

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(rf_model, X_test)

Let's focus on the first house in the test set.

In [ ]:
obs = 0
X_test.iloc[obs, :]

Using the above initiqlized `explainer` object, we will now calcualte SHAP values for all houses in the test set (to be able to compare the predictecd price of an individual house with the average prediction over all houses).

In [ ]:
shap_values = explainer(X_test)

Next, we plot the computed SHAP values for the first house as a `waterfall` plot

In [ ]:
shap.plots.waterfall(shap_values[obs])

The excellent `SHAP` (https://shap.readthedocs.io) package provides many more plots to visualize individual SHAP values and functions to aggregate local SHAP values to global SHAP values.